In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
import torch.nn.functional as F
import pandas as pd

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
from torch.utils.data import Dataset, DataLoader

train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

class LoadData(Dataset):
    def __init__(self, data):
        self.data = data.values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        inputs = self.data[index][1:]
        target = self.data[index][0]
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)
    
train_loader = DataLoader(LoadData(train_data), batch_size=32, shuffle=True, drop_last=True)
test_loader = DataLoader(LoadData(test_data), batch_size=32, shuffle=False, drop_last=True)

In [12]:


class RegressionModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

model = RegressionModel(90, 1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [22]:
# Обучение модели
n_epochs = 25
losses = []
for epoch in range(n_epochs):
    running_loss = 0.0
    model.train()
    for inputs, target in train_loader:
        inputs = inputs.to(device)
        target = target.to(device)
        optimizer.zero_grad() # обнуляем градиенты
        outputs = model(inputs) # прогноз модели
        loss = criterion(outputs, target) # вычисляем функцию потерь
        loss.backward() # вычисляем градиенты
        optimizer.step() # обновляем веса
        running_loss += loss.item()
    epoch_loss = running_loss/len(train_loader)
    losses.append(epoch_loss)
    print(f"Epoch {epoch}, Loss: {epoch_loss}")

    # Оценка модели на тестовых данных
    model.eval()
    test_loss = 0.0
    predictions = []
    with torch.no_grad():
        for inputs, target in test_loader:
            inputs = inputs.to(device)
            target = target.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, target)
            test_loss += loss.item()
            predictions.extend(outputs.detach().cpu().numpy())
    test_loss /= len(test_loader)
    print(f"MSE на тестовых данных: {test_loss}")

/home/noble6/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Loss: 11595.281039385729
MSE на тестовых данных: 11009.164330115686
Epoch 1, Loss: 11365.67304256112
MSE на тестовых данных: 8379.402155753894
Epoch 2, Loss: 11099.929851380566
MSE на тестовых данных: 8856.680216862605
Epoch 3, Loss: 10081.444152597582
MSE на тестовых данных: 10709.680728032039
Epoch 4, Loss: 9559.390051691542
MSE на тестовых данных: 7130.720383961995
Epoch 5, Loss: 8304.333707333222
MSE на тестовых данных: 6651.491065587753
Epoch 6, Loss: 6187.386672094445
MSE на тестовых данных: 5479.540368593656
Epoch 7, Loss: 3813.6533338324516
MSE на тестовых данных: 2249.836397097661
Epoch 8, Loss: 1457.0564546621715
MSE на тестовых данных: 707.8919444940029
Epoch 9, Loss: 423.9170335752558
MSE на тестовых данных: 443.5507587691148
Epoch 10, Loss: 162.52648906854296
MSE на тестовых данных: 452.35918355140933
Epoch 11, Loss: 153.95243315324282
MSE на тестовых данных: 449.44514170632914
Epoch 12, Loss: 155.0870006258448
MSE на тестовых данных: 395.84110866295987
Epoch 13, 

In [23]:
print(f"Финальное значение MSE для Model_1 ~50 эпох: {test_loss}")
print(model.parameters)

Финальное значение MSE для Model_1 ~50 эпох: 282.499281832423
<bound method Module.parameters of RegressionModel(
  (fc1): Linear(in_features=90, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)>


In [3]:
class RegressionModel2(nn.Module):
    def __init__(self, input_size, output_size):
        super(RegressionModel2, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(p=0.5)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        # x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        # x = self.dropout(x)
        x = self.fc3(x)
        return x
    
def train(model, train_loader, criterion, optimizer):
    running_loss = 0.0
    model.train()
    for inputs, target in train_loader:
        inputs = inputs.to(device)
        target = target.to(device)
        optimizer.zero_grad() # обнуляем градиенты
        outputs = model(inputs) # прогноз модели
        loss = criterion(outputs, target) # вычисляем функцию потерь
        loss.backward() # вычисляем градиенты
        optimizer.step() # обновляем веса
        running_loss += loss.item()
    epoch_loss = running_loss/len(train_loader)
    # losses.append(epoch_loss)
    print(f"Epoch {epoch}, Loss: {epoch_loss}")

def test(model, test_loader, criterion, optimizer):
    # Оценка модели на тестовых данных
    model.eval()
    test_loss = 0.0
    # predictions = []
    with torch.no_grad():
        for inputs, target in test_loader:
            inputs = inputs.to(device)
            target = target.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, target)
            test_loss += loss.item()
            # predictions.extend(outputs.detach().cpu().numpy())
    test_loss /= len(test_loader)
    print(f"MSE на тестовых данных: {test_loss}")

model_2 = RegressionModel2(90, 1)

model_2.to(DEVICE)

n_epochs = 25

train_loader = DataLoader(LoadData(train_data), batch_size=32, shuffle=True, drop_last=True)
test_loader = DataLoader(LoadData(test_data), batch_size=32, shuffle=False, drop_last=True)

for epoch in range(n_epochs):
    train(model_2, train_loader, criterion, optimizer)
    test(model_2, train_loader, criterion, optimizer)

NameError: name 'criterion' is not defined

| ID | GPU | MEM |
------------------
|  0 |  3% | 55% |


In [4]:
n_epochs = 3

early_stop_callback = pl.callbacks.early_stopping.EarlyStopping(
monitor='val_loss', # метрика для отслеживания
min_delta=0.00, # минимальное изменение метрики, чтобы считаться улучшением
patience=3, # количество эпох, в течение которых метрика не улучшалась, прежде чем обучение прервется
verbose=False, # печатать ли сообщения о прерывании обучения
mode='min' # определяет, какая метрика является "лучшей": "min" или "max"
)

trainer = pl.Trainer(
max_epochs=n_epochs,
callbacks=[early_stop_callback],
accelerator="gpu",
devices=1
)

model = RegressionModel().to(DEVICE)
print(model.parameters)
# model.train()
trainer.fit(model, train_loader, test_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 2.9 K 
1 | fc2  | Linear | 528   
2 | fc3  | Linear | 17    
3 | relu | ReLU   | 0     
--------------------------------
3.5 K     Trainable params
0         Non-trainable params
3.5 K     Total params
0.014     Total estimated model params size (MB)


<bound method Module.parameters of RegressionModel(
  (fc1): Linear(in_features=90, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)>
<bound method Module.parameters of RegressionModel(
  (fc1): Linear(in_features=90, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)>


Sanity Checking: 0it [00:00, ?it/s]

/home/noble6/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/tmp/ipykernel_117162/1676088828.py:27: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(outputs, targets)
/home/noble6/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of c

Training: 0it [00:00, ?it/s]

/tmp/ipykernel_117162/1676088828.py:20: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(outputs, targets)


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [10]:
trainer.logged_metrics

{'train_loss': tensor(15205.3066), 'val_loss': tensor(20474.4883)}